In [262]:
import random
def contradiction(kb):
    kb_updated = []

    for i in range(len(kb)):
        kb_updated.append(kb[i].replace("¬¬", ""))
        
    return kb_updated

In [263]:
def implication_elimnation(kb):
    kb_updated = []

    for statement in kb:
        if '∀' in statement or '∃' in statement:
            quantifier_part, rest = statement.split('(', 1)
            variable, inner_statement = rest.split(')', 1)
            variable = variable + ')'
            inner_statement = inner_statement.replace('→', '∨')
            inner_statement = inner_statement.strip()
            variable = '¬' + variable
            new_statement = f"{quantifier_part}({variable} {inner_statement}"
            kb_updated.append(new_statement)

        elif '→' in statement:
            parts = statement.split('→')
            kb_updated.append(f"(¬{parts[0].strip()} ∨ {parts[1].strip()})")

        else:
            kb_updated.append(statement)
            
    return kb_updated

In [264]:
def demorgan_law(kb):
    new_kb = []
    
    for predicate in kb:
        if "¬(" in predicate:
            not_index = predicate.index("¬(")
            
            open_count = 0
            close_index = -1
            for i in range(not_index + 1, len(predicate)):
                if predicate[i] == "(":
                    open_count += 1
                elif predicate[i] == ")":
                    if open_count == 1:
                        close_index = i
                        break
                    else:
                        open_count -= 1

            if close_index != -1:
                inside_predicate = predicate[not_index + 2:close_index]
           
                new_predicate = predicate[:not_index] + "("
                variable = ""
                for z in range(0, len(inside_predicate)):
                    
                    if inside_predicate[z] == "^":
                        if variable != "":
                            new_predicate += "¬" + variable
                            variable = ""
                        new_predicate += "∨"

                    elif inside_predicate[z] == "∨":
                        if variable != "":
                            new_predicate += "¬" + variable
                            variable = ""
                        new_predicate += "^"

                    elif inside_predicate[z] == "∀":
                        new_predicate += "∃" + inside_predicate[z + 1]
                        if variable != "":
                            new_predicate += "¬" + variable
                            variable = ""

                    elif inside_predicate[z]== "∃":
                        new_predicate += "∀" + inside_predicate[z + 1]
                        if variable != "":
                            new_predicate += "¬" + variable
                            variable = ""   
                        

                    elif inside_predicate[z] == "¬" or inside_predicate[z]  == " ":
                        if variable != "":
                            new_predicate += "¬" + variable
                            variable = ""
                        new_predicate += inside_predicate[z]

                    else:
                        if inside_predicate[z-1] == "∀" or inside_predicate[z-1] == "∃":
                            continue
                        else:
                            variable += inside_predicate[z]
                        
                if variable != "":
                    new_predicate += "¬" + variable

                new_predicate += ")" + predicate[close_index + 1:]
                new_kb.append(new_predicate)
    
    if new_kb != []:
        return new_kb
    else:
        return kb

In [265]:
def uniquantifiers_drop(kb):
    kb_updated = []
    
    for predicate in kb:
        new_predicate = predicate
        while new_predicate[0] == '∀':
            new_predicate = new_predicate.replace(new_predicate[0:2],"")
        kb_updated.append(new_predicate)

    return kb_updated

In [266]:
def move_quantifiers(kb):
    new_kb = []
    for predicate in kb:
        quantifiers = ""
        new_predicate = ""
        skip_next = False
        for i in range(len(predicate) - 1):
            if skip_next:
                skip_next = False
                continue
            if predicate[i] == '∀' or predicate[i] == '∃':
                quantifiers += predicate[i] + predicate[i + 1]
                skip_next = True

        for i in range(len(predicate) - 1):
            if skip_next:
                skip_next = False
                continue

            if predicate[i] == '∀' or predicate[i] == '∃':
                skip_next = True
            else:
                if i == (len(predicate) - 2):
                    new_predicate += predicate[i]
                    new_predicate += predicate[i+1]
                else:
                    new_predicate += predicate[i]

        new_kb.append(quantifiers + new_predicate)

    return new_kb

In [267]:
def skolemize(kb):
    kb_updated = []
    chars = ['H','M','N','C','D']
    for predicate in kb:
            new_predicate = kb
            if '∃' not in kb:
                return kb
            elif kb[0]=='∃':
                new_predicate = new_predicate.replace(kb[0:2],"")
                new_predicate = new_predicate.replace(kb[1],chars[random.randint(0, 4)])
            elif kb[0]=='∀':
                var1 = kb[1]
                var2 = kb[3]
                toReplace = "F(" + var1 + ")"
                new_predicate = new_predicate.replace(kb[2:5]," ")
                new_predicate = new_predicate.replace(kb[3],toReplace)

            kb_updated.append(new_predicate)


test = ["∀x∃y P(x) ∨ Q(y)"]
print(skolemize(test))

['∀x∃y P(x) ∨ Q(y)']


In [268]:
def vs_standardize(kb):
     kb_updated = []
     for predicate in kb:
          symbols = []
          predicate_holder = predicate
          
          for i in range (0,len(predicate_holder)-1):
               counter = 1
               new_symbol = ''

               if predicate_holder[i] == "∃" or  predicate_holder[i] == "∀":
                         new_symbol_code = -1

                         if predicate_holder[i+1] not in symbols:
                               symbols.append(predicate_holder[i+1])

                         else:
                              new_predicate = ""

                              while predicate_holder[i+1] in symbols:
                                   new_symbol_code = ord(predicate_holder[i+1]) + counter
                                   counter = counter + 1
                                   if new_symbol_code != -1:
                                        new_symbol = chr(new_symbol_code)
                                        symbols.append(new_symbol)
                                        break

                              for z in range (0, i+1):
                                   new_predicate = new_predicate + predicate_holder[z]
                              for j in range (i+1, len(predicate_holder)):
                                  if(predicate_holder[i+1] == predicate_holder[j]):
                                        new_predicate = new_predicate + new_symbol
                                  else:
                                        new_predicate = new_predicate + predicate_holder[j]

                              predicate_holder = new_predicate

          kb_updated.append(predicate_holder)
     return kb_updated     

In [269]:
def cnf(kb):
    kb_updated = []
    
    #1
    kb_updated = implication_elimnation(kb)
    #2
    kb_updated = demorgan_law(kb_updated)
    #3
    kb_updated = contradiction(kb_updated)
    #4
    kb_updated = vs_standardize(kb_updated)
    #5
    kb_updated = move_quantifiers(kb_updated)
    #6
    kb_updated = skolemize(kb_updated)
    #7
    kb_updated = uniquantifiers_drop(kb_updated)

    return kb_updated

In [270]:
kb = ["∀x(Student(x) → Attend(x))"
        ,"∃y∀x(Parent(x) ∨ Child(y))"
        ,"∀x(Student(z) → ¬¬Excellent(z))"
        ,"∀z(Cloudy(z)) ∨ ∀y (Depressed(y))"]

kb = cnf(kb)
for elem in kb:
    print(elem)

#(¬Student(x) ∨ Attend(x))
#∃y∀x(¬Parent(x) ∨ Child(y))
#(¬Student(z) ∨ Excellent(z))
#(¬Cloudy(z)) ∨ (Depressed(y))

(¬Student(x) ∨ Attend(x))
∃y∀x(¬Parent(x) ∨ Child(y))
(¬Student(z) ∨ Excellent(z))
(¬Cloudy(z) ) ∨  (Depressed(y))
